In [5]:
from src.loaders.datasetloader import DatasetLoader

In [6]:
rooms = DatasetLoader.run("../compute/dataset_20250130/test_generation")

In [7]:
rooms[1]

Room(id='graph_10', graph=[{'from': 'Wall_001', 'to': 'Wall_002', 'angle': 90.0}, {'from': 'Wall_002', 'to': 'Wall_001', 'angle': 90.0}, {'from': 'Wall_002', 'to': 'Wall_003', 'angle': -90.0}, {'from': 'Wall_003', 'to': 'Wall_002', 'angle': -90.0}, {'from': 'Wall_003', 'to': 'Wall_004', 'angle': -90.0}, {'from': 'Wall_004', 'to': 'Wall_003', 'angle': -90.0}, {'from': 'Wall_004', 'to': 'Wall_005', 'angle': -90.0}, {'from': 'Wall_005', 'to': 'Wall_004', 'angle': -90.0}, {'from': 'Wall_005', 'to': 'Wall_006', 'angle': -90.0}, {'from': 'Wall_006', 'to': 'Wall_005', 'angle': -90.0}, {'from': 'Wall_006', 'to': 'Wall_001', 'angle': -90.0}, {'from': 'Wall_001', 'to': 'Wall_006', 'angle': -90.0}, {'from': 'Window_001', 'to': 'Wall_004', 'uv_coordinates': {'u': 0.53158, 'v': 0.58074}}, {'from': 'Balcony_001', 'to': 'Wall_004', 'uv_coordinates': {'u': 0.0, 'v': 0.61742}}], walls=[Wall(id='Wall_001', center=Coordinate(x=0, y=0, z=0), bb=Bbox(width=0.37, height=4.45, length=0.4466), props=WallProps

In [8]:
from src.stats.stat_calculator import StatCalculator

In [10]:
res = StatCalculator.run(rooms)

In [11]:
res

[StatResult(mean=np.float64(5.351851851851852), max=np.int64(6), min=np.int64(4)),
 StatResult(mean=np.float64(1.0), max=np.int64(1), min=np.int64(1)),
 StatResult(mean=np.float64(2.0), max=np.int64(2), min=np.int64(2)),
 StatResult(mean=np.float64(28.32965842592593), max=np.float64(90.848), min=np.float64(5.23582))]